In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
train = pd.read_csv('data/train.csv', index_col=0)
train.head()

,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
ID,,,,,,,,,,,,,,,,,,,,,
000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,2200000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,2000000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [3]:
X = train.drop('target', axis=1)
y = train.target
del train

In [10]:
from sklearn.decomposition import FactorAnalysis

fa = FactorAnalysis(n_components=50, random_state=42)
X_fa = fa.fit_transform(X)

In [11]:
from sklearn.random_projection import SparseRandomProjection

srp = SparseRandomProjection(n_components=50, random_state=42)
X_srp = srp.fit_transform(X)

In [12]:
from sklearn.random_projection import GaussianRandomProjection

grp = GaussianRandomProjection(n_components=50, random_state=42)
X_grp = grp.fit_transform(X)

In [17]:
from sklearn.model_selection import train_test_split

X_added = pd.concat([
    pd.DataFrame(X_fa),
    pd.DataFrame(X_srp),
    pd.DataFrame(X_grp),
], axis=1)

y_log = np.log1p(y)

X_train, X_test, y_train, y_test = train_test_split(
    X_added, y_log, test_size=0.2, random_state=42
)

# X_val, X_test, y_val, y_test = train_test_split(
#     X_test, y_test, test_size=0.5, random_state=42
# )

print(X_train.shape, X_test.shape)

(3567, 150) (892, 150)


In [20]:
from sklearn.metrics.scorer import make_scorer

def rmsle_metric(y_test, y_pred): 
    assert len(y_test) == len(y_pred)
    y_test = np.exp(y_test)-1
    y_pred = np.exp(y_pred)-1
    rmsle = np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_test))**2))
    return rmsle

rmsle_scorer = make_scorer(rmsle_metric, greater_is_better=False)

In [21]:
from tpot import TPOTRegressor

tpot_config = {
    'xgboost.XGBRegressor': {
        'booster': ['gbtree', 'gblinear', 'dart'],
        'n_estimators': [100, 150],
        'max_depth': range(3, 15),
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5],
        'subsample': np.arange(0.05, 1.01, 0.05),
        'min_child_weight': range(1, 21),
        'n_jobs': [1]
    },
    'sklearn.neighbors.KNeighborsRegressor': {
        'n_neighbors': range(1, 51),
        'weights': ["uniform", "distance"],
        'p': [1, 2]
    },
    'sklearn.ensemble.RandomForestRegressor': {
        'n_estimators': [100],
        'max_features': np.arange(0.05, 1.01, 0.05),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21),
        'bootstrap': [True, False]
    },
}

model = TPOTRegressor(
    generations=20,
    population_size=100,
    n_jobs=2,
    verbosity=2,
    cv=5,
    early_stop=5,
    config_dict=tpot_config,
    scoring=rmsle_scorer
)
model.fit(X_train, y_train.values)

Optimization Progress:  10%|▉         | 201/2100 [1:25:32<14:48:44, 28.08s/pipeline]

Generation 1 - Current best internal CV score: -1.4293308407897003


Optimization Progress:  14%|█▍        | 302/2100 [2:30:28<15:52:50, 31.80s/pipeline]

Generation 2 - Current best internal CV score: -1.4293308407897003


Optimization Progress:  19%|█▉        | 402/2100 [3:43:44<19:11:42, 40.70s/pipeline]

...

Generation 7 - Current best internal CV score: -1.4269274198321136


Optimization Progress:  43%|████▎     | 905/2100 [10:36:34<11:22:58, 34.29s/pipeline]

Generation 8 - Current best internal CV score: -1.4269130150862304




TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestRegressor(RandomForestRegressor(input_matrix, bootstrap=False, max_features=0.15000000000000002, min_samples_leaf=2, min_samples_split=17, n_estimators=100), bootstrap=False, max_features=0.35000000000000003, min_samples_leaf=3, min_samples_split=12, n_estimators=100)


TPOTRegressor(config_dict={'sklearn.ensemble.RandomForestRegressor': {'min_samples_split': range(2, 21), 'n_estimators': [100], 'max_features': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]), 'min_samples_leaf': range(1, 21), ...NeighborsRegressor': {'weights': ['uniform', 'distance'], 'p': [1, 2], 'n_neighbors': range(1, 51)}},
       crossover_rate=0.1, cv=5, disable_update_check=False, early_stop=5,
       generations=20, max_eval_time_mins=5, max_time_mins=None,
       memory=None, mutation_rate=0.9, n_jobs=2, offspring_size=100,
       periodic_checkpoint_folder=None, population_size=100,
       random_state=None, scoring=None, subsample=1.0, verbosity=2,
       warm_start=False)

In [22]:
y_pred_t = model.predict(X_train)
print(rmsle_metric(y_train, y_pred_t))

y_pred = model.predict(X_test)
print(rmsle_metric(y_test, y_pred))

# y_pred_v = gbm.predict(X_val)
# print(rmsle_metric(y_val, y_pred_v))

0.09871870525336057
1.4040819942946092


In [43]:
test = pd.read_csv('data/test.csv', index_col=0)
test.head()

,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
ID,,,,,,,,,,,,,,,,,,,,,
000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00056a333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00056d8eb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
ids = test.reset_index()['ID']

In [45]:
from sklearn.decomposition import FactorAnalysis
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection

X_fa = fa.transform(test)

X_srp = srp.transform(test)

X_grp = grp.transform(test)

X_added = pd.concat([
    pd.DataFrame(X_fa),
    pd.DataFrame(X_srp),
    pd.DataFrame(X_grp),
], axis=1)

y_pred = gbm.predict(X_added)
y_pred

array([14.10120969, 14.89507214, 14.41820716, ..., 14.05671032,
       13.47478906, 15.4152626 ])

In [46]:
y_pred = np.exp(y_pred) - 1

In [47]:
y_pred[0]

1330691.0361930497

In [48]:
ids[0]

'000137c73'

In [50]:
pd.DataFrame(y_pred, index=ids, columns=['target']).to_csv('data/submit.csv')